# Main file for thesis project

Ie the top level script which runs the whole `pic -> recognise -> aim gimbal -> repeat` show

In [ ]:
#!/usr/bin/python3
# top level script to run the show

# TODO: add all the ncs stuff
# TODO: add all the gimbal control stuff
# TODO: add the EKF
# TODO: add the control stuff

from image_classifier import ImageClassifier
import gimbal_control as gc
from extended_kalman_filter import ExtendedKalmanFilter

classifier = ImageClassifier()  # done but not tested
EKF = ExtendedKalmanFilter()    # not done, not tested
SBGC = GimbalController()       # done, needs to be repackaged and tested

while True:
    pic = take_picture()

	obj_position = run_through_ncs(pic)
	
	# while the nn recognises the image (approx 80ms),
	# do other stuff:
	
	angles = get_angles_from_gimbal()

	states = EKF(obj_position, angles)
	
	control_action = controller(states)
	
	command_gimbal(control_action)


In [1]:
from image_classifier import ImageClassifier

[NbConvertApp] Converting notebook image_classifier.ipynb to python
[NbConvertApp] Writing 13187 bytes to image_classifier.py


In [2]:
IC = ImageClassifier(graph_file='../Models/MobileNet_SSD_caffe/graph',
                     label_file='../Models/MobileNet_SSD_caffe/categories.txt')

In [13]:
img_array = IC.take_picture_and_start_inference()
output_dict, bb, inference_time_ms = IC.get_inference_result()

print(output_dict)
print(bb)
print(inference_time_ms)

{'num_detections': 1, 'detection_scores_0': 97, 'detection_boxes_0': [(0, 128), (397, 576)], 'detection_classes_0': 15}
((128, 0), (576, 397))
79.94145


/usr/local/lib/python3.5/dist-packages/mvnc/mvncapi.py:418: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  tensor = numpy.fromstring(tensor.raw, dtype=numpy.float32)
